In [1]:
import torch
from torch import nn
import tools
import pandas
import data_process
import MyTransformer

In [2]:
data  = pandas.read_csv('./motionClassify.csv')
vocab = data_process.gen_vocab(data)
data_train  =  data_process.gen_dataset(data[:40000],vocab)
data_test = data_process.gen_dataset(data[40000:],vocab)
Batch_size = 64
train_iter = torch.utils.data.DataLoader(data_train,Batch_size,shuffle=True)
test_iter = torch.utils.data.DataLoader(data_test,Batch_size,shuffle=True)

In [3]:
num_hiddens, num_layers, dropout =  256, 2, 0.1#num_hidden 其实是embedding_size或者说embedding_dim
batch_size, num_steps = 64, 10
lr, num_epochs, device = 0.005, 300, tools.try_gpu()
key_size=query_size=value_size=ffn_num_input=norm_shape=num_hiddens
ffn_num_hiddens, num_heads =  64,4

In [8]:
class transformerEncoder(nn.Module):
    def __init__(self,vocab_size,key_size, query_size, value_size, num_hiddens,
    norm_shape, ffn_num_input, ffn_num_hiddens, num_heads,
    num_layers, dropout):
        super().__init__()
        self.encoder = MyTransformer.TransformerEncoder(
            vocab_size, key_size, query_size, value_size, num_hiddens,
            norm_shape, ffn_num_input, ffn_num_hiddens, num_heads,
            num_layers, dropout)
        self.dense = nn.Linear(num_hiddens,2)
    def forward(self,X):
        enc_X = self.encoder(X)
        return self.dense(enc_X[:,-1,:])

In [9]:
net = transformerEncoder(len(vocab),key_size,query_size,value_size,num_hiddens,
                         norm_shape,ffn_num_input,ffn_num_hiddens,num_heads,num_layers
                         ,dropout)

In [ ]:
optimizer = torch.optim.SGD(net.parameters(),lr)
tools.train(net,train_iter,device,optimizer,torch.nn.CrossEntropyLoss())

In [ ]:
tools.test(net,test_iter,device)